# Risco de ações

In [1]:
import pandas as pd
import numpy as np
import statistics
import math
from scipy import stats
import seaborn as srn
import matplotlib.pyplot as plt
srn.set()

In [17]:
dataset = pd.read_csv('acoes.csv')
dataset['Date'] = pd.to_datetime(dataset['Date'])
colunas = dataset.columns[1:]
print(dataset.shape)
dataset.head()

(1798, 7)


,Date,GOL,CVC,WEGE,MGLU,TOTS,BOVA
0,2015-01-02,14.99,15.20,5.923076,0.232812,11.910702,47.259998
1,2015-01-05,14.85,15.00,5.963461,0.237187,11.544731,46.320000
2,2015-01-06,15.21,14.80,5.875000,0.234062,10.822770,46.580002
3,2015-01-07,14.55,14.67,5.807692,0.241875,10.746248,48.150002
4,2015-01-08,14.27,14.15,5.905769,0.240000,10.995774,48.509998


In [3]:
print(dataset.info())
dataset.describe(include='all').T

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1798 entries, 0 to 1797
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    1798 non-null   object 
 1   GOL     1798 non-null   float64
 2   CVC     1798 non-null   float64
 3   WEGE    1798 non-null   float64
 4   MGLU    1798 non-null   float64
 5   TOTS    1798 non-null   float64
 6   BOVA    1798 non-null   float64
dtypes: float64(6), object(1)
memory usage: 98.5+ KB
None


,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Date,1798,1798,2015-12-16,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GOL,1798,NaN,NaN,NaN,15.7926,9.64064,1.16,7.7725,14.8,21.9,43.79
CVC,1798,NaN,NaN,NaN,30.1237,15.6602,6.10657,17.2,23.705,44.1925,64.8
WEGE,1798,NaN,NaN,NaN,15.4205,12.0484,4.94231,6.57692,9.07846,22.9038,46.105
MGLU,1798,NaN,NaN,NaN,7.06014,7.95957,0.030585,0.357997,3.92297,11.3244,27.45
TOTS,1798,NaN,NaN,NaN,16.7217,9.06115,7.1564,9.98104,11.578,24.4967,40.6
BOVA,1798,NaN,NaN,NaN,79.3597,23.8065,36.45,57.8,78.435,100,125.75


## Taxa de retorno anual

In [43]:
df_anual = pd.DataFrame()
taxa_retorno_log = lambda valor_inicial, valor_final: np.log(valor_final/valor_inicial)*100
anos = dataset['Date'].dt.year.unique()

In [55]:
# 2015 a 2022
for ano in anos:
    primeiro_dia = dataset[dataset['Date'].dt.year == ano]['Date'].min().date()
    ultimo_dia = dataset[dataset['Date'].dt.year == ano]['Date'].max().date()
    for coluna in colunas:
        valor_inicial = dataset.query('Date == @primeiro_dia')[coluna].values[0]
        valor_final = dataset.query('Date == @ultimo_dia')[coluna].values[0]
        taxa_retorno = taxa_retorno_log(valor_inicial, valor_final)
        df_anual.loc[coluna, f'taxa_retorno_{ano}'] = taxa_retorno
df_anual

,taxa_retorno_2015,taxa_retorno_2016,taxa_retorno_2017,taxa_retorno_2018,taxa_retorno_2019,taxa_retorno_2020,taxa_retorno_2021,taxa_retorno_2022
GOL,-178.312440,65.076790,117.472435,50.223057,36.878205,-39.902897,-34.140920,4.325629
CVC,-11.860573,63.734933,74.520037,20.420020,-33.287804,-77.587744,-40.745022,25.691039
WEGE,-2.965608,5.708249,46.793190,-9.282715,65.632628,76.512455,-12.336012,5.852815
MGLU,-121.692221,177.745281,184.209052,82.564540,71.868010,70.463576,-124.998910,4.082203
TOTS,-14.299333,-24.421777,23.670102,-10.480793,81.418491,21.992798,3.120818,27.376753
BOVA,-11.490413,34.856435,24.758641,11.911371,23.540034,0.358254,-12.384919,14.696275


## Variância

In [75]:
# por ação
taxas_gol = df_anual.loc['GOL',:].values
taxas_cvc = df_anual.loc['CVC',:].values
taxas_wege = df_anual.loc['WEGE',:].values
taxas_mglu = df_anual.loc['MGLU',:].values
taxas_tots = df_anual.loc['TOTS',:].values
taxas_bova = df_anual.loc['BOVA',:].values

# média
media_gol = taxas_gol.mean()
media_cvc = taxas_cvc.mean()
media_wege = taxas_wege.mean()
media_mglu = taxas_mglu.mean()
media_tots = taxas_tots.mean()
media_bova = taxas_bova.mean()

# variância
variancia_gol = taxas_gol.var()
variancia_cvc = taxas_cvc.var()
variancia_wege = taxas_wege.var()
variancia_mglu = taxas_mglu.var()
variancia_tots = taxas_tots.var()
variancia_bova = taxas_bova.var()